# UAV Tespiti için YOLOv11 Modeli

Bu Jupyter Notebook, YOLOv11 modelini kullanarak insansız hava araçlarını (UAV) tespit etmek için bir pipeline oluşturur. Aşağıdaki adımları içerir:
- **Eğitim**: Modeli veri setinizle eğitir.
- **Doğrulama**: Eğitilen modelin performansını değerlendirir.
- **Tahmin**: Yeni görüntüler üzerinde tahmin yapar.
- **Dışa Aktarma**: Modeli ONNX gibi formatlarda dışa aktarır.

Hiperparametreler bir yapılandırma dosyasında (YAML) tanımlanır ve kolayca değiştirilebilir. Türkçe açıklamalar, her adımı anlamanıza yardımcı olur.

## 1. Kütüphaneleri İçe Aktarma

Bu bölümde, gerekli Python kütüphanelerini içe aktarıyoruz:
- `ultralytics`: YOLO modelini çalıştırmak için.
- `yaml`: Yapılandırma dosyasını oluşturmak ve okumak için.
- `os`: Dosya ve dizin işlemleri için.

In [ ]:
from ultralytics import YOLO
import yaml
import os

## 2. Yapılandırma Ayarları

Hiperparametreler ve dosya yolları bir `config` sözlüğünde tanımlanır. Bu ayarlar, modelin nasıl eğitileceği, doğrulanacağı ve kullanılacağı gibi detayları içerir. Ayarları değiştirmek için bu bölümü düzenleyebilirsiniz.

**Önemli Alanlar**:
- `model`: Kullanılacak model mimarisi (örn. yolo11n) ve önceden eğitilmiş ağırlıklar.
- `training`: Eğitim için epoch sayısı, görüntü boyutu, öğrenme oranı gibi parametreler.
- `validation`: Doğrulama veri seti ve ayarları.
- `prediction`: Tahmin için kullanılacak görüntü ve eşik değerleri.
- `export`: Modelin dışa aktarılacağı format.

Yapılandırma, bir YAML dosyasına kaydedilir ve tekrar kullanılabilir.

In [ ]:
# Yapılandırma sözlüğü
config = {
    'model': {
        'architecture': 'yolo11n',  # Model: yolo11n, yolo11s, yolo11m vb.
        'pretrained': True,         # Önceden eğitilmiş ağırlıkları kullan
        'weights': 'yolo11n.pt',    # Önceden eğitilmiş ağırlık dosyası
        'custom_weights': '/home/furkanblt/PycharmProjects/UAV-Detection/runs/detect/train16/weights/best.pt'  # Özel model ağırlıkları
    },
    'training': {
        'data': '/home/furkanblt/PycharmProjects/UAV-Detection/Dataset/data.yaml',  # Veri seti YAML dosyası
        'epochs': 2,                # Eğitim için epoch sayısı
        'imgsz': 640,               # Görüntü boyutu (piksel)
        'batch_size': 16,           # Parti boyutu
        'optimizer': 'Adam',        # Optimizasyon algoritması
        'lr0': 0.001,               # Başlangıç öğrenme oranı
        'patience': 50,             # Erken durdurma sabrı
        'device': '',               # '0' GPU, '' otomatik, 'cpu' CPU için
        'augment': True             # Veri artırma
    },
    'validation': {
        'data': None,               # Doğrulama veri seti (None ise eğitim veri seti kullanılır)
        'imgsz': 640                # Doğrulama görüntü boyutu
    },
    'prediction': {
        'image_path': '/home/furkanblt/PycharmProjects/UAV-Detection/validations/bt2.jpg',  # Tahmin yapılacak görüntü
        'conf': 0.25,               # Güven eşiği
        'iou': 0.45,                # IoU eşiği (NMS için)
        'save': True,               # Sonuçları kaydet
        'save_path': 'validations/valid-data/result.jpg'  # Sonuçların kaydedileceği yol
    },
    'export': {
        'format': 'onnx',           # Dışa aktarma formatı: onnx, torchscript vb.
        'simplify': True            # ONNX modelini basitleştir
    }
}

# Yapılandırmayı YAML dosyasına kaydet
config_file = 'uav_detection_config.yaml'
with open(config_file, 'w') as f:
    yaml.dump(config, f)
print(f"Yapılandırma dosyası oluşturuldu: {config_file}")

## 3. Model Eğitimi

Bu bölümde, YOLOv11 modelini veri setinizle eğitiriz. Model, yapılandırma dosyasındaki ayarlara göre yüklenir ve eğitilir.

**Ne Yapılır**:
- Model, `yolo11n.yaml` dosyasından oluşturulur veya önceden eğitilmiş ağırlıklar yüklenir.
- Eğitim, belirtilen epoch sayısı, görüntü boyutu ve diğer hiperparametrelerle yapılır.
- Eğitim sonuçları döndürülür ve kaydedilir.

In [ ]:
def train_model(config):
    # Model ayarlarını al
    model_config = config['model']
    train_config = config['training']
    
    # Modeli yükle
    if model_config['pretrained']:
        model = YOLO(model_config['weights'])  # Önceden eğitilmiş modeli yükle
    else:
        model = YOLO(f"{model_config['architecture']}.yaml").load(model_config['weights'])  # YAML'dan oluştur ve ağırlıkları aktar

    # Modeli eğit
    print("Eğitim başlıyor...")
    results = model.train(
        data=train_config['data'],
        epochs=train_config['epochs'],
        imgsz=train_config['imgsz'],
        batch=train_config['batch_size'],
        optimizer=train_config['optimizer'],
        lr0=train_config['lr0'],
        patience=train_config['patience'],
        device=train_config['device'],
        augment=train_config['augment']
    )
    print("Eğitim tamamlandı!")
    return model, results

# Eğitimi çalıştır
trained_model, train_results = train_model(config)

## 4. Model Doğrulama

Eğitilen modelin performansı, doğrulama veri seti üzerinde test edilir. Bu adım, modelin ne kadar iyi genelleştiğini gösterir.

**Ölçümler**:
- mAP50-95: Ortalama hassasiyet (tüm IoU eşikleri).
- mAP50: IoU=0.5 için hassasiyet.
- mAP75: IoU=0.75 için hassasiyet.
- Kategorilere göre mAP değerleri.

In [ ]:
def validate_model(config, model=None):
    val_config = config['validation']
    model_config = config['model']
    
    # Modeli yükle (eğer sağlanmadıysa)
    if model is None:
        model = YOLO(model_config['custom_weights'])
    
    # Modeli doğrula
    print("Doğrulama başlıyor...")
    metrics = model.val(
        data=val_config['data'] or config['training']['data'],
        imgsz=val_config['imgsz']
    )
    
    # Sonuçları yazdır
    print(f"mAP50-95: {metrics.box.map:.4f}")
    print(f"mAP50: {metrics.box.map50:.4f}")
    print(f"mAP75: {metrics.box.map75:.4f}")
    print(f"Kategorilere göre mAP: {metrics.box.maps}")
    
    return metrics

# Doğrulamayı çalıştır
val_metrics = validate_model(config, trained_model)

## 5. Tahmin Yapma

Bu bölümde, eğitilen model yeni bir görüntü üzerinde tahmin yapar. Tahmin sonuçları, sınırlayıcı kutular (bounding boxes), sınıflar ve güven skorları içerir.

**Ne Yapılır**:
- Model, belirtilen görüntü üzerinde tahmin yapar.
- Sonuçlar ekranda gösterilir ve istenirse bir dosyaya kaydedilir.
- Sınırlayıcı kutuların koordinatları ve güven skorları yazdırılır.

In [ ]:
def predict_model(config, model=None):
    pred_config = config['prediction']
    model_config = config['model']
    
    # Modeli yükle (eğer sağlanmadıysa)
    if model is None:
        model = YOLO(model_config['custom_weights'])
    
    # Tahmin yap
    print("Tahmin yapılıyor...")
    results = model(
        pred_config['image_path'],
        conf=pred_config['conf'],
        iou=pred_config['iou'],
        stream=True
    )
    
    # Sonuçları işle
    for result in results:
        boxes = result.boxes
        xywh = boxes.xywh  # Merkez-x, merkez-y, genişlik, yükseklik
        xywhn = boxes.xywhn  # Normalleştirilmiş
        xyxy = boxes.xyxy  # Sol-üst-x, sol-üst-y, sağ-alt-x, sağ-alt-y
        xyxyn = boxes.xyxyn  # Normalleştirilmiş
        names = [result.names[cls.item()] for cls in boxes.cls.int()]  # Sınıf isimleri
        confs = boxes.conf  # Güven skorları
        
        # Sonuçları yazdır
        print(f"Tespit edilen nesneler: {names}")
        print(f"Güven skorları: {confs.tolist()}")
        
        # Kaydet veya göster
        if pred_config['save']:
            os.makedirs(os.path.dirname(pred_config['save_path']), exist_ok=True)
            result.save(filename=pred_config['save_path'])
            print(f"Sonuç kaydedildi: {pred_config['save_path']}")
        result.show()  # Ekranda göster
    
    return results

# Tahmini çalıştır
pred_results = predict_model(config, trained_model)

## 6. Modeli Dışa Aktarma

Eğitilen model, belirtilen formatta (örneğin ONNX) dışa aktarılır. Bu, modelin farklı platformlarda kullanılmasını sağlar.

**Ne Yapılır**:
- Model, yapılandırma dosyasındaki formata göre dışa aktarılır.
- ONNX için basitleştirme seçeneği kullanılır.

In [ ]:
def export_model(config, model=None):
    export_config = config['export']
    model_config = config['model']
    
    # Modeli yükle (eğer sağlanmadıysa)
    if model is None:
        model = YOLO(model_config['custom_weights'])
    
    # Modeli dışa aktar
    print("Model dışa aktarılıyor...")
    export_path = model.export(
        format=export_config['format'],
        simplify=export_config['simplify']
    )
    print(f"Model şu yola dışa aktarıldı: {export_path}")
    return export_path

# Dışa aktarmayı çalıştır
export_path = export_model(config, trained_model)

## 7. Hiperparametre Optimizasyonu

Modelin performansını artırmak için hiperparametreleri optimize edebilirsiniz. Aşağıdaki kod, farklı öğrenme oranları ve parti boyutları ile denemeler yapar.

**Nasıl Kullanılır**:
- `hyperparams` sözlüğünü düzenleyerek test etmek istediğiniz değerleri belirleyin.
- Her kombinasyon için model eğitilir ve doğrulanır.

In [ ]:
from itertools import product

# Test edilecek hiperparametreler
hyperparams = {
    'lr0': [0.0001, 0.001],
    'batch_size': [8, 16]
}

# Hiperparametre kombinasyonlarını dene
for lr, batch in product(hyperparams['lr0'], hyperparams['batch_size']):
    print(f"Deneme: lr0={lr}, batch_size={batch}")
    config['training']['lr0'] = lr
    config['training']['batch_size'] = batch
    
    # Modeli eğit ve doğrula
    model, results = train_model(config)
    metrics = validate_model(config, model)
    print(f"mAP50-95: {metrics.box.map:.4f}\n")

## Sonuç

Bu notebook, YOLOv11 modelini kullanarak UAV tespiti için kapsamlı bir pipeline sunar. Yapılandırma dosyasını düzenleyerek hiperparametreleri kolayca değiştirebilir, modeli eğitebilir, doğrulayabilir, tahmin yapabilir ve dışa aktarabilirsiniz.

**Öneriler**:
- Veri setinizin `data.yaml` dosyasında doğru yollar ve sınıflar tanımlı olduğundan emin olun.
- Daha iyi sonuçlar için `epochs` sayısını artırın (örn. 50-100).
- GPU kullanıyorsanız, `batch_size` ve `imgsz` değerlerini donanımınıza göre ayarlayın.
- Sorun yaşarsanız, hata mesajlarını paylaşarak destek alabilirsiniz!